# Udacity Self-Driving Car Project: Advanced Lane Finding
**By Elisa Chang**
___
#### The goals of this project are:
* To make the lane detection pipeline more robust to changes in lane color and changes in lighting conditions than when we first implemented the pipeline in the [CarND-LaneLines-P1](https://github.com/eeeja/CarND-LaneLines-P1) implementation.
* To calculate the curvature of the lanes so that we can adjust the steering wheel correctly.
___
#### We do this by using the following techniques:
Analysis:
1. [Correct Radial Distortion](#correct-radial-distortion): Compute and correct the camera calibration matrix and distortion coefficients given a set of chessboard images.
    * [Calibrate Camera](#calibrate-camera)
    * [Undistort Image](#undistort-image)
2. [Find Lane Lines](#find-lane-lines): Use color transforms & pixel gradients to detect lane line edges.
    * [ID Best Color Maps](#id-best-color-maps)
    * [Find Sobel Gradients](#find-sobel-gradients)
3. [Calculate Lane Line Curvature](#calculate-lane-line-curvature): Apply a perspective transform to rectify binary image ("birds-eye view").
    * [Calculate Birdseye Perspective](#calculate-birdseye-perspective)
    * [Detect lane pixels](#detect-lane-pixels)
    * [Determine Lane Curvature](#determine-lane-curvature)
    
Pipeline:
4. Process test images
5. Process test videos

This notebook explores in detail the python implementation of the lane detection pipeline. For a more analytical overview of the pipeline, please see [Writeup](https://github.com/eeeja/CarND-LaneLines-P1/blob/master/P1.ipynb) located at './Writeup.md'
___

### <a name="radial-distortion">Correct Radial Distortion</a>
Image Distortion happens when a 3D object is transformed to a 2D image. There are two forms of image distortion: Radial and perspective distortion. Here we focus on correcting the radial distortion. Further down at [Calculate Birdseye Perspective](#calculate-birdseye-perspective) we will adjust for the perspective distortion. 

Radial distortion happens when curved lenses in cameras bend light rays too much at the edge of lenses-- edges of images appear more/less curved than they actually are.

$$x_{corrected} = x(1+k_1r^2+k_2r^4+k_3r^6)$$
$$y_{corrected} = y(1+k_1r^2+k_2r^4+k_3r^6)$$

#### <a name="calibrate-camera">Calibrate Camera</a>
To fix radial distortion, first we begin by calibrating our camera. We do this by analyzing the checker images in the "./camera_cal" folder to determine what the camera matrix and distortion coefficients are. We need these values to understand how the camera lens distorts images when transforming from 3D objects to 2D images.

In [1]:
# Libraries used throughout pipeline
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Install imagio to use moviepy
import imageio
imageio.plugins.ffmpeg.download()

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os

In [ ]:
# Use cv2 functions findChessboardCorners() and drawChessboardCorners() 
# to find and draw corners of an imageimport numpy as np
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline

# Number of corners inside the checker image in X and Y dimensions
nx = 9# Manually counted corners within the image
ny = 6# Manually counted corners within the image

# Image 1: 
# Make a list of calibration images
# Edited image in Paint so it could have a white background
fname = './camera_cal/calibration6.jpg' 
img = cv2.imread(fname)

# Image 2:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Image 3:
# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
checker_img = img.copy()
# If found, draw corners
if ret == True:
    # Draw and display the corners
    cv2.drawChessboardCorners(checker_img, (nx, ny), corners, ret)
    plt.imshow(checker_img)

#### <a name="undistort-image">Undistort Image</a>
We need to run the above for all the checker images so that we can have a collection of image (distorted) points and object (undistorted) points. 

    "Image points" refer to the x,y,z coordinates of an object as displayed on the image. 
    "Object points" refer to the x,y,z coordinates of an object as displayed in real life.
    For both of these points, we assume the z coordinate is 0.

Using the same code from above, below we process all images in the *camera_cal* folder to collect the image and object points. Images with the detected corners are saved as "chessboardCorner_calibrationX.jpg" images in the **output_images** folder. Once all corner coordinates are collected, we run through all the images again and undistort them with `cv2.calibrateCamera` and `cv2.undistort.` These images are saved as "undistorted_calibrationX.jpg" images 

In [8]:
# Helper method used in our final pipeline later on

# Function that takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[0:2], None, None)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

# Helper function to get binary images
# zeros_like returns same dtype, assume all dtype is np.uint8 so we could use 0-255 range
def get_binary(img, thresh=(0,255)):
    binary = np.uint8(np.zeros_like(img))
    binary[(img>thresh[0]) & (img<=thresh[1])] = 255
    return binary

## All these methods assume the image has been converted to a gray-scale image
# Function applies Sobel x or y, then takes an absolute value and applies a threshold.
def abs_sobel_thresh(gray, orient='x', sobel_kernel=3, thresh=(0, 255)):
    if orient == 'x':
        # Need to set as cv2.CV_64F to capture both positive and negative slope
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a copy and apply the threshold
    binary_output = get_binary(scaled_sobel, thresh=thresh)
    
    return binary_output


# Function applies Sobel x and y, then computes the magnitude of the gradient
# and applies a threshold
def mag_thresh(gray, sobel_kernel=3, thresh=(0, 255)):
    # Take the gradient in x and y separately
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
    # Calculate the magnitude 
    abs_sobelxy = np.sqrt(0.1*sobely*sobely+sobelx*sobelx)
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    # Create a binary mask where mag thresholds are met
    binary_output = get_binary(scaled_sobel, thresh=thresh)
    
    return binary_output

# Define a function that applies Sobel x and y, then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Take the gradient in x and y separately
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    # Take the absolute value of the x and y gradients
    abs_sobelx = np.sqrt(sobelx*sobelx)
    abs_sobely = np.sqrt(sobely*sobely)
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad_dir =  np.arctan2(abs_sobely, abs_sobelx)
    
    # Create a binary mask where direction thresholds are met
    binary_output = get_binary(grad_dir,thresh=thresh)
    
    return binary_output

# Helper function that will take an image and return the R & G and Value gradients
def extract_lane_gradient(img,thresh=(0,255),dir_thresh=(0,1.5),ksize=13,RGB=0):
    """Input RGB Image"""
    HSV_img = img.copy()
    
    
    #Use only R & G values --> assume input image is RGB
    HSV_img[:,:,2] = HSV_img[:,:,0]
    
    if RGB ==0:
        HSV_img = cv2.cvtColor(HSV_img, cv2.COLOR_RGB2HSV)
        # Image 2: Grayscale of only R&G Values
        HSV_gray = HSV_img[:,:,2]
    
    else: 
        HSV_gray = cv2.cvtColor(HSV_img,cv2.COLOR_RGB2GRAY)
        HSV_gray[HSV_gray<140]=140


    # Image 3: x Gradient of only R&G Values
    HSV_gradx = abs_sobel_thresh(HSV_gray, orient='x', sobel_kernel=ksize, thresh=thresh)
    
    # Image 4: y Gradient of only R&G Values
    HSV_grady = abs_sobel_thresh(HSV_gray, orient='y', sobel_kernel=ksize, thresh=thresh)
    
    # Image 5: X & Y Gradient magnitude of only R&G Values
    HSV_mag_binary = mag_thresh(HSV_gray, sobel_kernel=ksize, thresh=thresh)
    
    # Image 6: Only gradients that are  almost verticle-- most likely to be lane lines
    HSV_dir_binary = dir_threshold(HSV_gray, sobel_kernel=ksize, thresh=dir_thresh)
    
    # Image 7: All gradients combined of only R&G Values 
    # Combine all gradients
    HSV_combined = np.zeros_like(HSV_dir_binary)
    HSV_combined[((HSV_gradx == 255) & (HSV_grady == 255)) 
                | ((HSV_mag_binary == 255) & (HSV_dir_binary == 255))] = 255
    
    # Ignore the bottom 100 pixels since they don't add useful information
    y_max = HSV_combined.shape[0]-50
    HSV_combined[y_max::]=0
    #HSV_combined = HSV_combined[0:y_max]
                
    return HSV_combined

# Perspective Transform helper function using the bright road image processed in the gradient section
# Passing in M because it is the same for all frames, no need to recalculate
def warper(image, M):
    offset = 100 # offset for dst points
    # Grab the image shape
    img_size = (image.shape[1], image.shape[0])

    # Warp the image using OpenCV warpPerspective()
    binary_warped = cv2.warpPerspective(image, M, img_size)
    return binary_warped

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    
import sys
from numpy import NaN, Inf, arange, isscalar, asarray, array

def peakdet(v, delta, x = None):
    """
    Converted from MATLAB script at http://billauer.co.il/peakdet.html
    
    Returns two arrays
    
    function [maxtab, mintab]=peakdet(v, delta, x)
    %PEAKDET Detect peaks in a vector
    %        [MAXTAB, MINTAB] = PEAKDET(V, DELTA) finds the local
    %        maxima and minima ("peaks") in the vector V.
    %        MAXTAB and MINTAB consists of two columns. Column 1
    %        contains indices in V, and column 2 the found values.
    %      
    %        With [MAXTAB, MINTAB] = PEAKDET(V, DELTA, X) the indices
    %        in MAXTAB and MINTAB are replaced with the corresponding
    %        X-values.
    %
    %        A point is considered a maximum peak if it has the maximal
    %        value, and was preceded (to the left) by a value lower by
    %        DELTA.
    
    % Eli Billauer, 3.4.05 (Explicitly not copyrighted).
    % This function is released to the public domain; Any use is allowed.
    
    """
    maxtab = []
    mintab = []
       
    if x is None:
        x = arange(len(v))
    
    v = asarray(v)
    
    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')
    
    if not isscalar(delta):
        sys.exit('Input argument delta must be a scalar')
    
    if delta <= 0:
        sys.exit('Input argument delta must be positive')
    
    mn, mx = Inf, -Inf
    mnpos, mxpos = NaN, NaN
    
    lookformax = True
    
    for i in arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True

    return array(maxtab)#, array(mintab)

# Pass a binary undistorted and warped image, now look at the histogram to get the lane coordinates

def get_lane_pix(binary_warped, left, right, nwindows=9, margin = 100, minpix =50):
    """
    binary_warped = undistarted and birdseye view warped image
    left and right = Line() objects
    nwindows = number of windows desired to split image in y direction when searching for lane pixels
    margin = x margin from window center to search for additinal lane pixels 
    minpix = min number of pixels needed be considered sufficent to update lane best x-coordinate 
    Returns an image with the lane lines and road highlighted and with radius of curvature and distance from center
    """
    #print(left.best_fit, right.best_fit)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped)) 
    
    # lane.bestx will be used as the center of the window to detect lane pixels
    if(left.bestx != None or right.bestx != None): #Use bestx from previous frame
        left.detected = True
        right.detected = True
    else: # No previous bestx, use histogram to find peaks
        # Take a histogram of the entire birds eye binary image
        histogram = np.sum(binary_warped, axis=0)
        midpoint = np.int(histogram.shape[0]/2)
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        #left.bestx = np.max(np.where(histogram[:midpoint]>20000))
        #left.bestx = np.argmax(histogram[:midpoint])
        #right.bestx = np.min(np.where(histogram[midpoint:]>20000))+midpoint
        #right.bestx = np.argmax(histogram[midpoint:])+midpoint
        max = peakdet(histogram, 5000)
        split1, split2 =np.split(max,2,axis=1)
        clearestLinex = split1[np.argmax(split2)]
        if clearestLinex < midpoint: # the left lane
            left.bestx = clearestLinex
            clearestLineOtherx = left.bestx + 700
            x_ind = (split1 > clearestLineOtherx-200) & (split1< clearestLineOtherx+200)
            if np.array(x_ind==0).all(): # No other lane detected, return function
                return out_img, left, right
            else:
                right.bestx = split1[np.where(split2==np.max(split2[x_ind]))]
        else:
            right.bestx = clearestLinex
            clearestLineOtherx = right.bestx - 700
            x_ind = (split1 > clearestLineOtherx-200) & (split1< clearestLineOtherx+200)
            if np.array(x_ind==0).all(): # No other lane detected, return function
                return out_img, left, right
            else:
                left.bestx = split1[np.where(split2==np.max(split2[x_ind]))]
            
    
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # recent_xfitted capture all best xs captured in current frame
    # Why bother storing this? 
    left.recent_xfitted = []
    right.recent_xfitted = []
        
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one and find all lane pixels
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = left.bestx - margin
        win_xleft_high = left.bestx + margin
        win_xright_low = right.bestx - margin
        win_xright_high = right.bestx + margin

        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Find all none zero pixels within window frame
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) 
                          & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) 
                           & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            # Formerly leftx_current
            left.bestx = np.int(np.mean(nonzerox[good_left_inds]))
            #temp_left_xfitted.append(left.bestx)
            left.recent_xfitted.append(left.bestx)          
        if len(good_right_inds) > minpix:        
            # Formerly rightx_current
            right.bestx = np.int(np.mean(nonzerox[good_right_inds]))
            #temp_right_xfitted.append(right.bestx)
            right.recent_xfitted.append(right.bestx)
   
    
    left.allx = np.concatenate(left_lane_inds)
    right.allx = np.concatenate(right_lane_inds)
   
    # Extract left and right line pixel positions
    leftx = nonzerox[left.allx]
    lefty = nonzeroy[left.allx] 
    rightx = nonzerox[right.allx]
    righty = nonzeroy[right.allx] 
    
    # If leftx or rightx is 0, then break, won't be able to completely ID road
    if len(leftx)==0 or len(rightx)==0:
        return out_img, left, right

    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    # Calculate radisu of curvature
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel
    
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    
    y_eval = binary_warped.shape[0]-1
    # Fit a second or first order polynomial to each
    # 30000 came from the observation that lanes with more than 30000 data points resulted in more accurate lines
    # If there are less than 30000 lines, then it might be better to find a straigh line equation
    if ( len(left.allx) > 30000): 
        left.best_fit = np.polyfit(lefty, leftx, 2)
        left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        left.radius_of_curvature = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    elif ( len(left.allx) > 10000 or left.best_fit ==None): # Otherwise use previous fit 
        left.best_fit = np.polyfit(lefty, leftx, 1)
        left.radius_of_curvature = None
    if ( len(right.allx) > 30000):
        right.best_fit = np.polyfit(righty, rightx, 2)
        right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
        right.radius_of_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    elif ( len(right.allx) > 10000 or right.best_fit ==None):  
        right.best_fit = np.polyfit(righty, rightx, 1)
        right.radius_of_curvature = None

#    if left.best_fit is None:
#        left.best_fit = temp_left_best_fit
#        right.best_fit = temp_right_best_fit
    
    # Not sure where to use this downstream
    #left.diffs = temp_left_best_fit - left.best_fit
    #right.diffs = temp_right_best_fit - right.best_fit
    
    # Update lane property
    #left.best_fit = temp_left_best_fit
    #right.best_fit = temp_right_best_fit     
    
#    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
#    left.radius_of_curvature = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
#    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
#    right.radius_of_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    
    #visualize
    if(len(left.best_fit)>2):
        left_fitx = left.best_fit[0]*ploty**2 + left.best_fit[1]*ploty + left.best_fit[2]
    else:
        left_fitx = left.best_fit[0]*ploty + left.best_fit[1]
    if(len(right.best_fit)>2):    
        right_fitx = right.best_fit[0]*ploty**2 + right.best_fit[1]*ploty + right.best_fit[2]
    else: 
        right_fitx = right.best_fit[0]*ploty + right.best_fit[1]
    #out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    #print("get_lane_pix out_img", out_img.shape)
    # Paint left and right lanes
    # out_img[nonzeroy[left.allx], nonzerox[left.allx]] = [255, 0, 0]
    # out_img[nonzeroy[right.allx], nonzerox[right.allx]] = [0, 0, 255]
    ## Fill in between the lines
    # Get all x values that are in between the lines: 
    
    x_ind =np.array([[x for x in range(binary_warped.shape[1]) if x >= left_fitx[y] and x <= right_fitx[y]] 
          for y in range(binary_warped.shape[0])])
    
    for row in range(binary_warped.shape[0]):
        out_img[np.asarray(np.ones(len(x_ind[row]))*row, np.int32),np.asarray(x_ind[row])]=[0,255,255]

    # Black out non colored pixels
    r, g, b = 255, 255, 255
    rgb_thresh=[r,g,b]
    
    thresholds = ((out_img[:,:,0] == rgb_thresh[0]) \
    & (out_img[:,:,1] == rgb_thresh[1]) \
    & (out_img[:,:,2] == rgb_thresh[2])) \
    | ((out_img[:,:,0] < rgb_thresh[0]) \
    & (out_img[:,:,1] < rgb_thresh[1]) \
    & (out_img[:,:,2] < rgb_thresh[2]))
    out_img[thresholds] = [0,0,0]
    
    
    
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, binary_warped.shape[1])
    #plt.ylim(binary_warped.shape[0], 0)
    return out_img, left,right

def weighted_img(img, initial_img, α=0.8, β=1.0, λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    # Add back the bottom frame we removed
    # Additional extracted vectors:
    #add_y = np.zeros((initial_img.shape[0]-img.shape[0],img.shape[1]),dtype=np.uint8)
    #img = np.append(img,np.dstack((add_y, add_y, add_y)),axis = 0)
    
    return cv2.addWeighted(initial_img, α, img, β, λ)

    
def get_vehiclepos(img_width, left_lane, right_lane):
    # Define conversions in x and y from pixels space to meters
    xm_per_pix = 3.7/700# meters per pixel in x dimension
    
    # get the first window
    lane_center = (left_lane.bestx+ right_lane.bestx)/2
    img_center = img_width/2
    
    # If lane_center > img_center, then to the right, and is X meters right of lane
    diff = img_center - lane_center
    diff = diff*xm_per_pix
    
    left_lane.line_base_pos = diff
    right_lane.line_base_pos = diff
    
    return diff, left_lane, right_lane


# write radius of curvature and vehicle position on 
def write_text(img, left, right):
    img_shape = img.shape
    if ((left.radius_of_curvature == None) and (right.radius_of_curvature==None)):
        radius = 0
    elif left.radius_of_curvature == None:
        radius = right.radius_of_curvature
    elif right.radius_of_curvature == None:
        radius = left.radius_of_curvature
    else:
        radius = np.mean([left.radius_of_curvature, right.radius_of_curvature])

    pos, left, right = get_vehiclepos(img_shape[1],left,right)
    
    if pos >= 0:
        direction = "right"
    else:
        direction = "left"
    lane_pos = np.abs(pos)

    radius_text = "Radius of Curvature = %.2f (m)" % radius
    lane_pos_text = "Vehicle is %.2f meters %s of center" % (lane_pos,direction) 
    cv2.putText(img, radius_text, (50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(img, lane_pos_text, (50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    return img

In [3]:
# Retrieve all images
# This loop might take some time since it is iterating and processing all camera_cal images
# Need to run this to get imgpoints and objpoints
import glob
import os
images = glob.glob('./camera_cal/calibration*.jpg')

# Number of corners inside the checker image in X and Y dimensions
nx = 9# Manually counted corners within the image
ny = 6# Manually counted corners within the image

# Create output folder if it doesn't exist
output_folder = './output_images'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Camera Calibration variables to fix optical distortion from lens
objpoints = [] #3D points in real world space
imgpoints = [] #2D points on image

# Returns an array of object points
# Z dim will always be 0
objp = np.zeros((nx*ny, 3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2) #x, y coordinates

# Read in all images in camera_cal folder
# Collect imgpoints to use to undistort
for fname in images:
    img = cv2.imread(fname)

    # Step 1: Calibrate Camera 
    # We will use the camera calibrations detected from the checker images to determine the 
    # intrinsic camera properties which we will use downstream for our car camera images to 
    # undistort the video stream.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

    # If found, draw corners
    if ret == True:
        # Append relevant points to use later on when undistorting images
        imgpoints.append(corners)
        objpoints.append(objp)
        checkered_img = img.copy()

        # Draw and display the corners
        #cv2.drawChessboardCorners(checkered_img, (nx, ny), corners, ret)
        
        # Save id'd points to output folder
        #save_fname = os.path.join(output_folder,'chessBoardCorners_' + os.path.basename(fname))
        #cv2.imwrite(save_fname, checkered_img)

# Undistort Image
#for fname in images:
#    img = cv2.imread(fname)

    # Step 2: Correct for Distortion 
#    undistorted = cal_undistort(img, objpoints, imgpoints)
#    save_fname = os.path.join(output_folder,'undistorted_' + os.path.basename(fname))

#    cv2.imwrite(save_fname, undistorted)
    

In [ ]:
#Find average diestances between peaks
images = glob.glob('./C/frame*.JPG')
## Project Variables
#left_x = []
#right_x = []
#diff = []

## Challenge
#left_Cx = []
#right_Cx = []
#diff_C = []

## Harder Challenge
left_HCx = []
right_HCx = []
diff_HC = []

for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    left = Line()
    right = Line()
    img_size = [img.shape[1],img.shape[0]]
    src = np.float32(
        [[(0 + 100), img_size[1]],
        [(img_size[0]/2) - 150, (img_size[1]*3 / 5)+75],
        [(img_size[0]/2) + 150, (img_size[1]*3 / 5)+75],
        [(img_size[0] - 100), (img_size[1])]])

    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32(
        [[(img_size[0] / 6), img_size[1]],
        [(img_size[0] / 6), 0],
        [(img_size[0] * 5 / 6), 0],
        [(img_size[0] * 5 / 6), img_size[1]]])

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # First undistort radial distortions
    undistort = cal_undistort(img, objpoints, imgpoints)
    #print("undistort.shape", undistort.shape)

    # Run through a guassian smoothing
    gaussian = cv2.GaussianBlur(undistort,(7,7),0)
    #print("gaussian.shape", gaussian.shape)

    # Extract R & G colors and HSV Value channel
    gradient = extract_lane_gradient(gaussian,thresh=(20,255),dir_thresh=(0.7,1.3),RGB=1)
    #print("gradient.shape", gradient.shape)

    # Get lane colored birdseye perspective
    #lane = gradient.copy()
    binary_warped = warper(gradient, M)
    #print("binary_warped.shape", binary_warped.shape)

    histogram = np.sum(binary_warped, axis=0)
    midpoint = int(binary_warped.shape[1]/2)
    left_HCx.append( np.argmax(histogram[:midpoint]))
    
    right_HCx.append(np.argmax(histogram[midpoint:]))
from scipy import stats
diff_HC =left_HCx-right_HCx
print(np.mean(diff_HC),stats.mode(diff_HC),np.median(diff_HC))

In [ ]:
from scipy import stats
right_HCx = np.ones(len(right_HCx))*midpoint+np.array(right_HCx)
diff_HC =np.array(left_HCx)-np.array(right_HCx)
print(len(right_HCx),len(left_HCx))
print(np.mean(diff_HC),stats.mode(diff_HC),np.median(diff_HC))
print(left_HCx[0],right_HCx[0])

Here is an example of an original input image vs the undistorted output the above code generated. Notice how you can see some minor radial distortion on the right hand side where the edge of the board curves inside. While the right hand side of the undistorted image is straighter than the original image.

In [ ]:
# Display sample distorted and undistorted images
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3))
f.tight_layout()
img = cv2.imread('./camera_cal/calibration4.jpg')

undistorted = cv2.imread('./output_images/undistorted_calibration4.jpg')
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(undistorted)
ax2.set_title('Undistorted', fontsize=20)
save_fname = os.path.join(output_folder,'orig_vs_undistorted_' + os.path.basename(fname))
f.savefig(save_fname, bbox_inches='tight')

In [ ]:
# Replicate with a light road image
#fnameLte = './test_images/test4.jpg'
fnameLte = 'frame1000.jpg'
imgLte_BGR = cv2.imread(fnameLte)
imgLte_RGB = cv2.cvtColor(imgLte_BGR, cv2.COLOR_BGR2RGB)

# Correct for Distortion 
undistorted_lte = cal_undistort(imgLte_RGB, objpoints, imgpoints)
# Save image for use downstream
save_fname = os.path.join(output_folder,'undistorted_' + os.path.basename(fnameLte))
cv2.imwrite(save_fname, undistorted_lte)

# Compare before and after
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,10))
f.tight_layout()
ax1.imshow(imgLte_RGB)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(undistorted_lte)
ax2.set_title('Undistorted', fontsize=20)
save_fname = os.path.join(output_folder,'orig_vs_undistorted_' + os.path.basename(fnameLte))
f.savefig(save_fname, bbox_inches='tight')

### <a name="find-lane-lines">Find Lane Lines</a>
Now that we have adjusted for some of the optical distortions caused my the camera lens, we can begin manipulating the images to find the lane lines. 

Lane lines can be hard to detect in different lighting conditions and when they are different colors. Below we observe what color maps and gradients best bring out yellow and white lane lines in shadows and in the light.

#### <a name="id-best-color-maps">ID Best Color Maps</a>
Once we extract the specific channel we want to view, I run the image through a threshold filter to only pick up on pixels of a certain range of threshold. We had to modify the threshold value for the light vs dark road images. As the pipeline gets further refined downstream, these thresholds would have to be automatically adjusted as the lighting conditions change on the road.

**RGB color map on bright & dark roads:** 

*Analysis:* 
* The *B* (blue) values cannot capture yellow lines well.
* The *R* value seems to capture yellow and white lane lines well in both light and dark roads

In [ ]:
# Gradient Thresholds
threshLte = (140,255)
threshDrk = (40,100)

# Visualize RGB channels of undistorted test4.jpg-- "light road" image
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(undistorted_lte)
ax1.set_title('RGB: Light Road', fontsize=20)
r_binary = get_binary(undistorted_lte[:,:,(0)], thresh=threshLte)
ax2.imshow(r_binary, cmap = 'gray')
ax2.set_title('R', fontsize=20)
g_binary = get_binary(undistorted_lte[:,:,(1)], thresh=threshLte)
ax3.imshow(g_binary, cmap = 'gray')
ax3.set_title('G', fontsize=20)
g_binary = get_binary(undistorted_lte[:,:,(2)], thresh=threshLte)
ax4.imshow(g_binary, cmap = 'gray')
ax4.set_title('B', fontsize=20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Save dark image "additionalDarkRoadTest.jpg" for use downstream
fnameDrk = './test_images/additionalDarkRoadTest.jpg'

# cv2 reads in image as BGR
imgDrk_BGR = cv2.imread(fnameDrk)
imgDrk_RGB = cv2.cvtColor(imgDrk_BGR, cv2.COLOR_BGR2RGB)
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(imgDrk_RGB)
ax1.set_title('RGB: Dark Image', fontsize=20)
r_binary = get_binary(imgDrk_RGB[:,:,(0)], thresh=threshDrk)
ax2.imshow(r_binary, cmap = 'gray')
g_binary = get_binary(imgDrk_RGB[:,:,(1)], thresh=threshDrk)
ax3.imshow(g_binary, cmap = 'gray')
g_binary = get_binary(imgDrk_RGB[:,:,(2)], thresh=threshDrk)
ax4.imshow(g_binary, cmap = 'gray')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

**HSV color map on bright & dark roads:** HSV stands for hue, saturation, and value. Read more on how the color representation is calculated [here](http://codeitdown.com/hsl-hsb-hsv-color/)

*Analysis:* 
* The *value* field seems to be best at identifying white and yellow lines for both well lit and dark images

In [ ]:
threshLte=(50,255)
imgLte_HSV = cv2.cvtColor(undistorted_lte,cv2.COLOR_RGB2HSV)
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(imgLte_HSV)
ax1.set_title('HVS: Light Road', fontsize=20)

h_binary = get_binary(imgLte_HSV[:,:,(0)],thresh=threshLte)
ax2.imshow(h_binary, cmap='gray')
ax2.set_title('H', fontsize=20)

v_binary = get_binary(imgLte_HSV[:,:,(2)],thresh=threshLte)
ax3.imshow(v_binary, cmap='gray')
ax3.set_title('V', fontsize=20)

s_binary = get_binary(imgLte_HSV[:,:,(1)],thresh=threshLte)
ax4.imshow(s_binary, cmap='gray')
ax4.set_title('S', fontsize=20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Dark Image
imgDrk_HSV = cv2.cvtColor(imgDrk_BGR,cv2.COLOR_BGR2HSV)
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(imgDrk_HSV)
ax1.set_title('HVS: Dark Road', fontsize=20)

h_binary = get_binary(imgDrk_HSV[:,:,(0)],thresh=threshDrk)
ax2.imshow(h_binary, cmap='gray')

v_binary = get_binary(imgDrk_HSV[:,:,(2)],thresh=threshDrk)
ax3.imshow(v_binary, cmap='gray')

s_binary = get_binary(imgDrk_HSV[:,:,(1)],thresh=threshDrk)
ax4.imshow(s_binary, cmap='gray')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

**HLS color map on bright and dark road:** HLS stands for hue, lightness, and saturation. 

*Analysis:*
* The *saturation* field is able to capture both white and yellow lines. This is because the saturation value in *HLS* is depenedent on the associated lightness of the pixel, and not just its intensity. 
* The *lightness* dimension doesn't bring out the lane lines as much as the *value* dimension in the *HSV* space. This might be because *lightness* is an average of the brightest and darkest RGB values, while the *value* is simply the brightest value. 
* Using the existing light (220,255) and dark (40,100) gradient thresholds, the HLS slices are less able to pick up lane lines.

In [ ]:
# HLS Analysis
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
imgLte_HLS = cv2.cvtColor(undistorted_lte, cv2.COLOR_RGB2HLS)
ax1.imshow(imgLte_HLS)
ax1.set_title('HLS: Light Road', fontsize=20)

h_binary = get_binary(imgLte_HLS[:,:,(0)],thresh=threshLte)
ax2.imshow(h_binary, cmap='gray')
ax2.set_title('H', fontsize=20)

l_binary = get_binary(imgLte_HLS[:,:,(1)],thresh=threshLte)
ax3.imshow(l_binary, cmap='gray')
ax3.set_title('L', fontsize=20)

s_binary = get_binary(imgLte_HLS[:,:,(2)],thresh=threshLte)
ax4.imshow(s_binary, cmap='gray')
ax4.set_title('S', fontsize=20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Dark road
imgDrk_HLS = cv2.cvtColor(imgDrk_BGR,cv2.COLOR_BGR2HLS)
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(imgDrk_HLS)
ax1.set_title('HLS: Dark Road', fontsize=20)

h_binary = get_binary(imgDrk_HLS[:,:,(0)],thresh=threshDrk)
ax2.imshow(h_binary, cmap='gray')

l_binary = get_binary(imgDrk_HLS[:,:,(1)],thresh=threshDrk)
ax3.imshow(l_binary, cmap='gray')

s_binary = get_binary(imgDrk_HLS[:,:,(2)],thresh=threshDrk)
ax4.imshow(s_binary, cmap='gray')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

**Color map conclusions:**
* Specifically to bring out white and yellow lanes, we should ignore the blue values
* Within the *HVS* color map, the *value* field is best in both bright and dark lighting situations at bringing out white and yellow lanes.
* WIthin the *HLS* color map, the *saturation* field brings out lane lines best in bright images, but is not as reliable in detecting white lanes in dark lighting. 

Using these observations, it seems like we could maximize our lane detection if we do not use the blue channel and if we use the *value* field in the HVS color map. 

___
#### <a name="find-sobel-gradients">Find Sobel Gradients</a>
The sobel operator is a [convolution](http://setosa.io/ev/image-kernels/) matrix which is used to approximate the changes in pixel intensity (gradient). A 3x3 kernel looks like: 

$G_x = 
\left[ \begin{array}{cccc}
1 & 0 & -1 \\
2 & 0 & -2 \\
1 & 0 & -1 \\ \end{array} \right] * A$

$G_y = 
\left[ \begin{array}{cccc}
1 & 2 & 1 \\
0 & 0 & 0 \\
-1 & -2 & -1 \\ \end{array} \right] * A$

The key properties of the sobel operator is that it is depending on if you're look at the x or y gradient, the kernel has symmetric but opposite direction values in the first half/top half of the matrix to the second half/bottom half. This causes the convolution to return 0 if the pixels in the surrounding area are similar and will return larger values otherwise, consequently highlighting edges. Below we look at the edges detected using the sobel operators on a normal image, then on an image that has been enhanced leveraging our observations from analysis on different color maps.

We use the sobel operator here rather than the canny edge detection algorithm so that we can focus on finding edges of a certain direction (more vertical rather than horizontal) so that we could customize that edge detection process to only return edges most likely to be lane lines. 
___
**Sobel Operators on original image**
Below we collect different gradients. In the next section, we will compare gradients extracted from the original image vs gradients extracted from an image with the color map manipulated.

**Sobel Operator on Image with manipulated Color map**
The cvtColor documentation states that the alorithm used to convert RGB to color is
$$Y = 0.299 R + 0.587 G + 0.114 B$$
Knowing this, we will replace the $B$ value with $R$, since the red value seemed to bring out the lane lines best from the above RGB on dark and bright road sections.

In [ ]:
gray = cv2.cvtColor(undistorted_lte,cv2.COLOR_RGB2GRAY)
gray[gray<140]=140
plt.imshow(gray)

In [ ]:
# Compare a few different combinations of gradients. 
# Goal is to select combination that best highlights lane lines
# Reset threshLte otherwise threshold too high for these color maps
threshLte=(20,255)
ksize =15
### Set 1: Gradients of Original Image
# Image 1: Original test4 image
# Image 2: Grayscale
pre_gray = undistorted_lte.copy()
pre_gray[:,:,2]=pre_gray[:,:,0]
gray = cv2.cvtColor(pre_gray,cv2.COLOR_RGB2GRAY)
gray[gray<200]=0

# Image 3: x Gradient
gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=threshLte)

# Image 4: y Gradient
grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=threshLte)

# Image 5: X & Y Gradient magnitude
mag_binary = mag_thresh(gray, sobel_kernel=ksize, thresh=threshLte)

# Image 6: Only gradients that are  almost verticle-- most likely to be lane lines
dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0.7, 1.3))

# Image 7: All gradients combied
# Combine all gradients
combined = np.zeros_like(dir_binary)
combined[((gradx == 255) & (grady == 255)) | ((mag_binary == 255) & (dir_binary == 255))] = 255

###---------------------------------------------
### Set 2: Gradients of image with only Red & Green channel values 
# Image 1: Copy R values over B channel
img_RG = undistorted_lte.copy()
img_RG[:,:,2] = img_RG[:,:,0]

# Image 2: Grayscale of only R&G Values
gray_RG = cv2.cvtColor(img_RG,cv2.COLOR_RGB2GRAY)
gray_RG[gray_RG<200]=0

# Image 3: x Gradient of only R&G Values
gradx_RG = abs_sobel_thresh(gray_RG, orient='x', sobel_kernel=ksize, thresh=threshLte)

# Image 4: y Gradient of only R&G Values
grady_RG = abs_sobel_thresh(gray_RG, orient='y', sobel_kernel=ksize, thresh=threshLte)

# Image 5: X & Y Gradient magnitude of only R&G Values
mag_binary_RG = mag_thresh(gray_RG, sobel_kernel=ksize, thresh=threshLte)

# Image 6: Only gradients that are  almost verticle-- most likely to be lane lines
dir_binary_RG = dir_threshold(gray_RG, sobel_kernel=ksize, thresh=(0.7, 1.3))

# Image 7: All gradients combined of only R&G Values 
# Combine all gradients
combined_RG = np.zeros_like(dir_binary_RG)
combined_RG[((gradx_RG == 255) & (grady_RG == 255)) | ((mag_binary_RG == 255) & (dir_binary_RG == 255))] = 255

###---------------------------------------------
### Set 3: Gradients of V channel of HSV image converted from image with only Red & Green channel values
# Image 1: Only R&G Values
img_RG_V = img_RG.copy()
img_RG_V = cv2.cvtColor(img_RG_V, cv2.COLOR_RGB2HSV)

#img_RG_V[:,:,0]= img_RG_V[:,:,2]
#img_RG_V[:,:,1]= img_RG_V[:,:,2]

# Image 2: Grayscale of only R&G and V value
gray_RG_V = img_RG_V[:,:,0]

# Image 3: x Gradient of only R&G Values
gradx_RG_V = abs_sobel_thresh(gray_RG_V, orient='x', sobel_kernel=ksize, thresh=threshLte)

# Image 4: y Gradient of only R&G Values
grady_RG_V = abs_sobel_thresh(gray_RG_V, orient='y', sobel_kernel=ksize, thresh=threshLte)

# Image 5: X & Y Gradient magnitude of only R&G Values
mag_binary_RG_V = mag_thresh(gray_RG_V, sobel_kernel=ksize, thresh=threshLte)

# Image 6: Only gradients that are  almost verticle-- most likely to be lane lines
dir_binary_RG_V = dir_threshold(gray_RG_V, sobel_kernel=ksize, thresh=(0.7, np.pi/2))

# Image 7: All gradients combined of only R&G Values 
# Combine all gradients
combined_RG_V = np.zeros_like(dir_binary_RG_V)
combined_RG_V[((gradx_RG_V == 255) & (grady_RG_V == 255)) | ((mag_binary_RG_V == 255) & (dir_binary_RG_V == 255))] = 255


#plot all original images
fig = plt.figure(figsize = (18,30))
ax1 = fig.add_subplot(731)
ax1.set_title('Original Image', fontsize=20)
ax1.imshow(undistorted_lte)
ax4 = fig.add_subplot(734)
ax4.set_title('Grayscale', fontsize=20)
ax4.imshow(gray, cmap='gray')
ax7 = fig.add_subplot(737)
ax7.set_title('x Gradient', fontsize=20)
ax7.imshow(gradx, cmap = 'gray')
ax10 = fig.add_subplot(7,3,10)
ax10.set_title('Y Gradient', fontsize=20)
ax10.imshow(grady, cmap = 'gray')
ax13 = fig.add_subplot(7,3,13)
ax13.set_title('X & Y Gradient Magnitude', fontsize=20)
ax13.imshow(mag_binary,cmap='gray')
ax16 = fig.add_subplot(7,3,16)
ax16.set_title('Vertical Gradients', fontsize=20)
ax16.imshow(dir_binary,cmap='gray')
ax19 = fig.add_subplot(7,3,19)
ax19.set_title('All Gradients Combined', fontsize=20)
ax19.imshow(combined,cmap='gray')

#plot all images with manipulated color spalce
ax2 = fig.add_subplot(732)
ax2.set_title('Only R & G values', fontsize=20)
ax2.imshow(img_RG)
ax5 = fig.add_subplot(735)
ax5.imshow(gray_RG, cmap='gray')
ax8 = fig.add_subplot(738)
ax8.imshow(gradx_RG, cmap = 'gray')
ax11 = fig.add_subplot(7,3,11)
ax11.imshow(grady_RG, cmap = 'gray')
ax14 = fig.add_subplot(7,3,14)
ax14.imshow(mag_binary_RG,cmap='gray')
ax17 = fig.add_subplot(7,3,17)
ax17.imshow(dir_binary_RG,cmap='gray')
ax20 = fig.add_subplot(7,3,20)
ax20.imshow(combined_RG,cmap='gray')

#plot all images with manipulated color spalce
ax3 = fig.add_subplot(733)
ax3.set_title('V (HSV) from RG channels', fontsize=20)
ax3.imshow(img_RG_V)
ax6 = fig.add_subplot(736)
ax6.imshow(gray_RG_V, cmap='gray')
ax9 = fig.add_subplot(739)
ax9.imshow(gradx_RG_V, cmap = 'gray')
ax12 = fig.add_subplot(7,3,12)
ax12.imshow(grady_RG_V, cmap = 'gray')
ax15 = fig.add_subplot(7,3,15)
ax15.imshow(mag_binary_RG_V,cmap='gray')
ax18 = fig.add_subplot(7,3,18)
ax18.imshow(dir_binary_RG_V,cmap='gray')
ax21 = fig.add_subplot(7,3,21)
ax21.imshow(combined_RG_V,cmap='gray')

In [ ]:
# Repeat above with image of road with dark lighting
###### Find best threshold parameters
threshDr = (20,255)
# Image 1: Original (imgDrk_RGB)
# Image 2: Grayscale
gray_Drk = cv2.cvtColor(imgDrk_RGB,cv2.COLOR_RGB2GRAY)
gray_Drk[gray_Drk<140]=0

# Image 3: x Gradient
gradx_Drk = abs_sobel_thresh(gray_Drk, orient='x', sobel_kernel=ksize, thresh=threshDrk)

# Image 4: y Gradient
grady_Drk = abs_sobel_thresh(gray_Drk, orient='y', sobel_kernel=ksize, thresh=threshDrk)

# Image 5: X & Y Gradient magnitude
mag_binary_Drk = mag_thresh(gray_Drk, sobel_kernel=ksize, thresh=threshDrk)

# Image 6: Only gradients that are  almost verticle-- most likely to be lane lines
dir_binary_Drk = dir_threshold(gray_Drk, sobel_kernel=ksize, thresh=(0.7, 1.3))

# Image 7: All gradients combied
# Combine all gradients
combined_Drk = np.zeros_like(dir_binary_Drk)
combined_Drk[((gradx_Drk == 255) & (grady_Drk == 255)) | ((mag_binary_Drk == 255) & (dir_binary_Drk == 255))] = 255

###---------------------------------------------
### Set 2: Gradients of image with only Red & Green channel values 
# Image 1: Copy R values over B channel
img_RG_Drk = imgDrk_RGB.copy()
img_RG_Drk[:,:,2] = img_RG_Drk[:,:,0]

# Image 2: Grayscale of only R&G Values
gray_RG_Drk = cv2.cvtColor(img_RG_Drk,cv2.COLOR_RGB2GRAY)
gray_RG_Drk[gray_RG_Drk<200]=0

# Image 3: x Gradient of only R&G Values
gradx_RG_Drk = abs_sobel_thresh(gray_RG_Drk, orient='x', sobel_kernel=ksize, thresh=threshDrk)

# Image 4: y Gradient of only R&G Values
grady_RG_Drk = abs_sobel_thresh(gray_RG_Drk, orient='y', sobel_kernel=ksize, thresh=threshDrk)

# Image 5: X & Y Gradient magnitude of only R&G Values
mag_binary_RG_Drk = mag_thresh(gray_RG_Drk, sobel_kernel=ksize, thresh=threshDrk)

# Image 6: Only gradients that are  almost verticle-- most likely to be lane lines
dir_binary_RG_Drk = dir_threshold(gray_RG_Drk, sobel_kernel=ksize, thresh=(0.7, 1.3))

# Image 7: All gradients combined of only R&G Values 
# Combine all gradients
combined_RG_Drk = np.zeros_like(dir_binary_RG_Drk)
combined_RG_Drk[((gradx_RG_Drk == 255) & (grady_RG_Drk == 255)) 
                | ((mag_binary_RG_Drk == 255) & (dir_binary_RG_Drk == 255))] = 255

###---------------------------------------------
### Set 3: Gradients of V channel of HSV image converted from image with only Red & Green channel values
# Image 1: Only R&G Values
img_RG_V_Drk = img_RG_Drk.copy()
img_RG_V_Drk = cv2.cvtColor(img_RG_V_Drk, cv2.COLOR_RGB2HSV)
img_RG_V_Drk[:,:,0]= img_RG_V_Drk[:,:,2]
img_RG_V_Drk[:,:,1]= img_RG_V_Drk[:,:,2]

# Image 2: Grayscale of only R&G and V value
gray_RG_V_Drk = img_RG_V_Drk[:,:,2]

# Image 3: x Gradient of only R&G Values
gradx_RG_V_Drk = abs_sobel_thresh(gray_RG_V_Drk, orient='x', sobel_kernel=ksize, thresh=threshDrk)

# Image 4: y Gradient of only R&G Values
grady_RG_V_Drk = abs_sobel_thresh(gray_RG_V_Drk, orient='y', sobel_kernel=ksize, thresh=threshDrk)

# Image 5: X & Y Gradient magnitude of only R&G Values
mag_binary_RG_V_Drk = mag_thresh(gray_RG_V_Drk, sobel_kernel=ksize, thresh=threshDrk)

# Image 6: Only gradients that are  almost verticle-- most likely to be lane lines
dir_binary_RG_V_Drk = dir_threshold(gray_RG_V_Drk, sobel_kernel=ksize, thresh=(0.7, np.pi/2))

# Image 7: All gradients combined of only R&G Values 
# Combine all gradients
combined_RG_V_Drk = np.zeros_like(dir_binary_RG_V_Drk)
combined_RG_V_Drk[((gradx_RG_V_Drk == 255) & (grady_RG_V_Drk == 255)) 
                  | ((mag_binary_RG_V_Drk == 255) & (dir_binary_RG_V_Drk == 255))] = 255


#plot all original images
fig = plt.figure(figsize = (18,30))
ax1 = fig.add_subplot(731)
ax1.set_title('Original Image', fontsize=20)
ax1.imshow(imgDrk_RGB)
ax4 = fig.add_subplot(734)
ax4.set_title('Grayscale', fontsize=20)
ax4.imshow(gray_Drk, cmap='gray')
ax7 = fig.add_subplot(737)
ax7.set_title('x Gradient', fontsize=20)
ax7.imshow(gradx_Drk, cmap = 'gray')
ax10 = fig.add_subplot(7,3,10)
ax10.set_title('Y Gradient', fontsize=20)
ax10.imshow(grady_Drk, cmap = 'gray')
ax13 = fig.add_subplot(7,3,13)
ax13.set_title('X & Y Gradient Magnitude', fontsize=20)
ax13.imshow(mag_binary_Drk,cmap='gray')
ax16 = fig.add_subplot(7,3,16)
ax16.set_title('Vertical Gradients', fontsize=20)
ax16.imshow(dir_binary_Drk,cmap='gray')
ax19 = fig.add_subplot(7,3,19)
ax19.set_title('All Gradients Combined', fontsize=20)
ax19.imshow(combined_Drk,cmap='gray')

#plot all images with manipulated color spalce
ax2 = fig.add_subplot(732)
ax2.set_title('Only R & G values', fontsize=20)
ax2.imshow(img_RG_Drk)
ax5 = fig.add_subplot(735)
ax5.imshow(gray_RG_Drk, cmap='gray')
ax8 = fig.add_subplot(738)
ax8.imshow(gradx_RG_Drk, cmap = 'gray')
ax11 = fig.add_subplot(7,3,11)
ax11.imshow(grady_RG_Drk, cmap = 'gray')
ax14 = fig.add_subplot(7,3,14)
ax14.imshow(mag_binary_RG_Drk,cmap='gray')
ax17 = fig.add_subplot(7,3,17)
ax17.imshow(dir_binary_RG,cmap='gray')
ax20 = fig.add_subplot(7,3,20)
ax20.imshow(combined_RG_Drk,cmap='gray')

#plot all images with manipulated color spalce
ax3 = fig.add_subplot(733)
ax3.set_title('V (HSV) from RG channels', fontsize=20)
ax3.imshow(img_RG_V_Drk)
ax6 = fig.add_subplot(736)
ax6.imshow(gray_RG_V_Drk, cmap='gray')
ax9 = fig.add_subplot(739)
ax9.imshow(gradx_RG_V_Drk, cmap = 'gray')
ax12 = fig.add_subplot(7,3,12)
ax12.imshow(grady_RG_V_Drk, cmap = 'gray')
ax15 = fig.add_subplot(7,3,15)
ax15.imshow(mag_binary_RG_V_Drk,cmap='gray')
ax18 = fig.add_subplot(7,3,18)
ax18.imshow(dir_binary_RG_V_Drk,cmap='gray')
ax21 = fig.add_subplot(7,3,21)
ax21.imshow(combined_RG_V_Drk,cmap='gray')

In [ ]:
len(np.mean(cv2.cvtColor(imgDrk_RGB,cv2.COLOR_RGB2GRAY),axis = 0))
R = np.array([1117,1110,1079,1095,1101,1089,1012,1096,986,989,1050,1180])
L = np.array([326,330,292,273,312,319,322,326,236,240,300,444])
print(len(R),len(L), R-L)

**Gradient conclusions:**
* In both the dark and light images, the "Combined" image (last image) in the "HSV" column (the third column) extract the lane edges the best.

In the light image, the horizontal edge of the hood is also detected strongly. However, since that will be a consistent line and since it is not part of the road we can easily filter that out downstream so it doesn't affect the lane calculations.

In the final pipeline, we will make sure to only use *Red* and *Green* fields of an image and focus on the *Value* component of the HSV channel using the `extract_lane_gradient` function. 

## <a name='calculate-lane-line-curvature'>Calculate Lane Line Curvature</a>
Now we transform the edges we detected into a birdseye perspective so we can more accurately measure the curve of the road.
___
* ** Tangential/Perspective Distortion: **
Objects in images look slanted because camera is not aligned perfectly parallel to the imaging plane.
(Source Udacity Self Driving Care Notes)
$$x_{corrected} = x+[2p_1xy+p_2(r^2+2x^2)]$$
$$y_{corrected} = y+[p_1(r^2+2y^2)+2p_2xy]$$

Below we transform the road to a birdseye view perspective so we could better understand how the flat road is curving.

In [ ]:
# Test warped function:
combined_RG_V = extract_lane_gradient(imgLte_RGB, thresh=(20,255), dir_thresh=(0.7,1.3)) 

offset = 100 # offset for dst points
# Grab the image shape
img_size = (combined_RG_V.shape[1], combined_RG_V.shape[0])

# For source points I'm grabbing the outer four detected corners
src = np.float32(
    [[(0 + 100), img_size[1]],
    [(img_size[0]/2) - 150, (img_size[1]*3 / 5)+75],
    [(img_size[0]/2) + 150, (img_size[1]*3 / 5)+75],
    [(img_size[0] - 100), (img_size[1])]])
    
# For destination points, I'm arbitrarily choosing some points to be
# a nice fit for displaying our warped result 
# again, not exact, but close enough for our purposes
dst = np.float32(
    [[(img_size[0] / 6), img_size[1]],
    [(img_size[0] / 6), 0],
    [(img_size[0] * 5 / 6), 0],
    [(img_size[0] * 5 / 6), img_size[1]]])
# Given src and dst points, calculate the perspective transform matrix
M = cv2.getPerspectiveTransform(src, dst)
# Warp the image using OpenCV warpPerspective()
warped = combined_RG_V.copy()
warped = cv2.warpPerspective(warped, M, img_size)

src_int = np.array(src,np.int32).reshape((-1,1,2))

# Draw src polygon lines
img = imgLte_RGB.copy()
cv2.polylines(img, [src_int],True,color=(255,0,0),thickness=5)

# Draw gradient src lines
combined_RG_V_copy = combined_RG_V.copy()
cv2.polylines(combined_RG_V_copy, [src_int],True,color=(255,0,0),thickness=5)

# Draw Birdseye perspective src lines
cv2.rectangle(warped,tuple(dst[0]),tuple(dst[2]),(255,0,0), thickness = 5)

# Visualize changes
fig = plt.figure(figsize=(16,5))
ax1 = fig.add_subplot(131)
ax1.set_title('Original Image', fontsize=20)
ax1.imshow(img)
ax2 = fig.add_subplot(132)
ax2.set_title('Binary Gradient', fontsize=20)
ax2.imshow(combined_RG_V_copy, cmap = 'gray')
ax3 = fig.add_subplot(133)
ax3.set_title('Birdseye Warped', fontsize=20)
ax3.imshow(warped, cmap = 'gray')
fig.savefig('./output_images/orig_vs_birdseye_warped', bbox_inches='tight')

In [ ]:
# Put extract_lane_gradient() and warper() together
binary_warped = extract_lane_gradient(imgLte_RGB, thresh=(20,255), dir_thresh=(0.7,1.3)) 
binary_warped = warper(binary_warped, M)
plt.imshow(binary_warped, cmap = 'gray')

In [ ]:
#histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
histogram = np.sum(binary_warped, axis=0)
plt.plot(histogram)

## Finding Lanes
We now use a histogram to find the peaks of where white pixels are to determine where lane lines are.


In [ ]:
left_lane = Line()
right_lane = Line()
result,left_lane, right_lane = get_lane_pix(binary_warped,left_lane,right_lane)
plt.imshow(result)

In [ ]:
# Visualize Result
# Create an image to draw on and an image to show the selection window
#def draw_lanes():
out_img = np.dstack((binary_warped, binary_warped, binary_warped))
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

In [4]:
# Test Pipeline on all images in the test_images folder
test_image_folder = "test_images/"
test_image_output_folder = "output_images/"

video_output_folder = "output_videos/"

# Create output folder if it doesn't exist
if not os.path.exists(video_output_folder):
    os.makedirs(video_output_folder)

# Create output folder if it doesn't exist
if not os.path.exists(test_image_output_folder):
    os.makedirs(test_image_output_folder)
    
# Helper method to process image:
def process_image(img):
    global left_lane
    global right_lane
    img_size = (img.shape[1], img.shape[0])
    # For source points I'm grabbing the outer four detected corners
    src = np.float32(
    [[(0 + 100), img_size[1]],
    [(img_size[0]/2) - 150, (img_size[1]*3 / 5)+75],
    [(img_size[0]/2) + 150, (img_size[1]*3 / 5)+75],
    [(img_size[0] - 100), (img_size[1])]])

    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32(
        [[(img_size[0] / 6), img_size[1]],
        [(img_size[0] / 6), 0],
        [(img_size[0] * 5 / 6), 0],
        [(img_size[0] * 5 / 6), img_size[1]]])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # First undistort radial distortions
    undistort = cal_undistort(img, objpoints, imgpoints)
    
    # Run through a guassian smoothing
    gaussian = cv2.GaussianBlur(undistort,(7,7),0)
    
    # Extract R & G colors and HSV Value channel
    gradient = extract_lane_gradient(gaussian,thresh=(20,255),dir_thresh=(0.7,1.3),ksize = 13,RGB=1)
    
    # Get lane birdseye perspective
    lane = warper(gradient, M)

    # Extract lane centers and fill in detected road
    detected_lane, left_lane, right_lane = get_lane_pix(lane,left_lane,right_lane)
    
    # Draw back to original image
    unwarped = warper(detected_lane, Minv)

    # Cast over the original Image
    original = weighted_img(unwarped, img)
        
    result = write_text(original, left_lane, right_lane)
         
    # Calculate Lane Curvature
    return result

#for filename in os.listdir(test_image_folder):
    
#    img = cv2.imread(os.path.join(test_image_folder,filename))
#    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#    output = process_image(img)
    
    # Uncomment the following code if you are running the code locally and wish to save the image
#    mpimg.imsave(os.path.join(test_image_output_folder,"%s" %(filename)), output)

In [ ]:
img = cv2.imread('frameC53.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

left = Line()
right = Line()
img_size = [img.shape[1],img.shape[0]]
src = np.float32(
    [[(0 + 100), img_size[1]],
    [(img_size[0]/2) - 150, (img_size[1]*3 / 5)+75],
    [(img_size[0]/2) + 150, (img_size[1]*3 / 5)+75],
    [(img_size[0] - 100), (img_size[1])]])

# For destination points, I'm arbitrarily choosing some points to be
# a nice fit for displaying our warped result 
# again, not exact, but close enough for our purposes
dst = np.float32(
    [[(img_size[0] / 6), img_size[1]],
    [(img_size[0] / 6), 0],
    [(img_size[0] * 5 / 6), 0],
    [(img_size[0] * 5 / 6), img_size[1]]])

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
# First undistort radial distortions
undistort = cal_undistort(img, objpoints, imgpoints)
print("undistort.shape", undistort.shape)

# Run through a guassian smoothing
gaussian = cv2.GaussianBlur(undistort,(7,7),0)
print("gaussian.shape", gaussian.shape)

# Extract R & G colors and HSV Value channel
gradient = extract_lane_gradient(gaussian,thresh=(20,255),dir_thresh=(0.7,1.3),RGB=1)
print("gradient.shape", gradient.shape)

# Get lane colored birdseye perspective
#lane = gradient.copy()
binary_warped = warper(gradient, M)
print("binary_warped.shape", binary_warped.shape)

result,left, right = get_lane_pix(binary_warped,left,right)
#left, right = get_lane_pix(binary_warped,left,right)
#print(left.bestx, right.bestx,left.best_fit,right.best_fit,len(right.allx))
#print("result.shape", result.shape)


# ***********************************************


#unwarped = warper(result, Minv)
#print("unwarped", unwarped.shape)

# Cast over the original Image
#original = weighted_img(unwarped, img)
#print("original", original.shape)

#result = write_text(original,left_lane,right_lane)
#print("result", result.shape)
#mpimg.imsave("test_text.jpg", result)

fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(161)
ax1.set_title('Original Image', fontsize=20)
ax1.imshow(img)
ax2 = fig.add_subplot(162)
ax2.set_title('undistort', fontsize=20)
ax2.imshow(undistort)
ax3 = fig.add_subplot(163)
ax3.set_title('Gaussian', fontsize=20)
ax3.imshow(gaussian)
ax4 = fig.add_subplot(164)
ax4.set_title('Gradient', fontsize=20)
ax4.imshow(gradient)
ax5 = fig.add_subplot(165)
ax5.set_title('binary_warped', fontsize=20)
ax5.imshow(binary_warped)
#ax6 = fig.add_subplot(166)
#ax6.set_title('result', fontsize=20)
#ax6.imshow(original)


In [ ]:
nwindows=9
margin = 100
minpix =50
"""
binary_warped = undistarted and birdseye view warped image
left and right = Line() objects
nwindows = number of windows desired to split image in y direction when searching for lane pixels
margin = x margin from window center to search for additinal lane pixels 
minpix = min number of pixels needed be considered sufficent to update lane best x-coordinate 
Returns an image with the lane lines and road highlighted and with radius of curvature and distance from center
"""
#print(left.best_fit, right.best_fit)
# Create an output image to draw on and  visualize the result
out_img = np.dstack((binary_warped, binary_warped, binary_warped)) 
histogram = np.sum(binary_warped, axis=0)
midpoint = np.int(histogram.shape[0]/2)
# lane.bestx will be used as the center of the window to detect lane pixels
if(left.bestx != None or right.bestx != None): #Use bestx from previous frame
    left.detected = True
    right.detected = True
else: # No previous bestx, use histogram to find peaks
    # Take a histogram of the entire birds eye binary image
    histogram = np.sum(binary_warped, axis=0)
    midpoint = np.int(histogram.shape[0]/2)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    #left.bestx = np.max(np.where(histogram[:midpoint]>20000))
    #left.bestx = np.argmax(histogram[:midpoint])
    #right.bestx = np.min(np.where(histogram[midpoint:]>20000))+midpoint
    #right.bestx = np.argmax(histogram[midpoint:])+midpoint
    max = peakdet(histogram, 5000)
    split1, split2 =np.split(max,2,axis=1)
    clearestLinex = split1[np.argmax(split2)]
    if clearestLinex < midpoint: # the left lane
        left.bestx = clearestLinex
        clearestLineOtherx = left.bestx + 700
        x_ind = (split1 > clearestLineOtherx-50) & (split1< clearestLineOtherx+50)
        if np.array(x_ind==0).all(): # No other lane detected, return function
            #return out_img, left, right
            print('pass1')
        else:
            right.bestx = split1[np.where(split2==np.max(split2[x_ind]))]
    else:
        right.bestx = clearestLinex
        clearestLineOtherx = right.bestx - 700
        x_ind = (split1 > clearestLineOtherx-50) & (split1< clearestLineOtherx+50)
        if np.array(x_ind==0).all(): # No other lane detected, return function
            #return out_img, left, right
            print('pass2')
        else:
            left.bestx = split1[np.where(split2==np.max(split2[x_ind]))]
print(left.bestx, right.bestx)
plt.plot(histogram)
print(np.argmax(histogram[:midpoint]))
print(np.argmax(histogram[midpoint:]))
print(midpoint)

In [ ]:


# Run pipeline for videos
# Create video Output folder
video_output_folder = "output_videos/"

# Create output folder if it doesn't exist
if not os.path.exists(video_output_folder):
    os.makedirs(video_output_folder)
project_video = "%sproject_video.mp4" % (video_output_folder)

left_lane = Line()
right_lane = Line()

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
project = clip1.fl_image(process_image) #NOTE: this function expects color images!!
#%time 
project.write_videofile(project_video, audio=False)

In [6]:
clip1 = VideoFileClip("challenge_video.mp4")
#video = clip1.show()(process_image) #NOTE: this function expects color images!!
project_video = "%schallenge_video.mp4" % (video_output_folder)
left_lane = Line()
right_lane = Line()
video = clip1.fl_image(process_image)

#%time 
video.write_videofile(project_video, audio=False)

[MoviePy] >>>> Building video output_videos/challenge_video.mp4
[MoviePy] Writing video output_videos/challenge_video.mp4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 485/485 [39:44<00:00,  4.74s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video.mp4 



In [ ]:
# Debugging, save each frame
#import cv2
vidcap = cv2.VideoCapture("harder_challenge_video.mp4")
success,image = vidcap.read()
count = 0
success = True
#while success or count < 10:
while count < 10:
  success,image = vidcap.read()
  #print('Read a new frame: %s' % success)
  #cv2.imwrite("frameHC%d.jpg" % count, image)     # save frame as JPEG file
  count += 1

In [7]:
clip1 = VideoFileClip("harder_challenge_video.mp4")
left_lane = Line()
right_lane = Line()
project_video = "%sharder_challenge_video.mp4" % (video_output_folder)
white_clip = clip1.fl_image(process_image).subclip(0,5) #NOTE: this function expects color images!!
#%time 
white_clip.write_videofile(project_video, audio=False)

[MoviePy] >>>> Building video output_videos/harder_challenge_video.mp4
[MoviePy] Writing video output_videos/harder_challenge_video.mp4


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 125/126 [10:44<00:05,  5.07s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/harder_challenge_video.mp4 

